#### Detect model from Yu, et al
Implementation of the detection model from the Yu, et al [paper](https://drive.google.com/file/d/1nYl4w41CAcj8XwTEdVwcD5lVheUFIHVy/view?usp=sharing)

In [ ]:
# !pip install pickle5

In [ ]:
# import libraries
import tensorflow as tf
tf.__version__
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, losses, optimizers, callbacks

import os
import numpy as np
import pandas as pd

import helpers as helper
from keras_model_s3_wrapper import *

import boto3
import pickle5 as pickle
s3 = boto3.resource('s3')
bucket_name = 'wafer-capstone'
my_bucket = s3.Bucket(bucket_name)

In [ ]:
# List out objects in bucket
for my_bucket_object in my_bucket.objects.all():
    print(my_bucket_object.key)

In [ ]:
# specify variables for model
path = 'data'
result_path = 'results'
model_path = 'saved_models'

filename = 'WM-clean-paper'
option = '-detund'
map_column = 'waferMap224'

model_id = 'yudetect'
data_id = 'paper'
note = '' # -optional

In [ ]:
# load train, dev, and test sets
# directly from S3
start = time.time()

train_key = f'{path}/{filename}-train{option}.pkl'
dev_key = f'{path}/{filename}-dev.pkl'
test_key = f'{path}/{filename}-test.pkl'

train = pickle.loads(bucket_name.Object(train_key).get()['Body'].read())
dev = pickle.loads(bucket_name.Object(dev_key).get()['Body'].read())
test = pickle.loads(bucket_name.Object(test_key).get()['Body'].read())

print("Wall time: {:.2f} seconds".format(time.time() - start))
print(f"Train: {len(train)}")
print(f"Dev: {len(dev)}")
print(f"Test: {len(test)}")

print(f"Sanity check: {np.unique(train[map_column][0]),}")

In [ ]:
# # load train, dev, and test sets
# # directly from S3
# start = time.time()

# s3 = boto3.client('s3')
# bucket_name = 'wafer-capstone'

# obj = s3.get_object(Bucket = bucket_name, Key = f'{path}/{filename}-train{option}.pkl')
# body = obj['Body'].read()
# train = pickle.loads(body)

# obj = s3.get_object(Bucket = bucket_name, Key = f'{path}/{filename}-dev.pkl')
# body = obj['Body'].read()
# dev = pickle.loads(body)

# obj = s3.get_object(Bucket = bucket_name, Key = f'{path}/{filename}-test.pkl')
# body = obj['Body'].read()
# test = pickle.loads(body)

# print("Wall time: {:.2f} seconds".format(time.time() - start))
# print(f"Train: {len(train)}")
# print(f"Dev: {len(dev)}")
# print(f"Test: {len(test)}")

# print(f"Sanity check: {np.unique(train[map_column][0]),}")

In [ ]:
# # load train, dev, and test sets
# # from local instance
# start = time.time()

# with open(f'{path}/{filename}-train{option}.pkl', "rb") as fh:
#     train = pickle.load(fh)
# with open(f'{path}/{filename}-dev.pkl', "rb") as fh:
#     dev = pickle.load(fh)
# with open(f'{path}/{filename}-test.pkl', "rb") as fh:
#     test = pickle.load(fh)

# print("Wall time: {:.2f} seconds".format(time.time() - start))
# print(f"Train: {len(train)}")
# print(f"Dev: {len(dev)}")
# print(f"Test: {len(test)}")

#### Quick EDA

In [ ]:
# baseline accuracy of test set
nones = len(test[test.failureType == 'none'])
total = len(test)
print(f"Baseline accuracy: {nones/total*100:.2f}%")

In [ ]:
# train failure type distribution
helper.defect_distribution(train, note='Train Set')

In [ ]:
# dev failure type distribution
helper.defect_distribution(dev, note='Dev Set')

In [ ]:
# test failure type distribution
helper.defect_distribution(test, note='Test Set')

#### Data set-up

In [ ]:
# prepare inputs
start = time.time()

x_train = np.stack(train[map_column])
x_val = np.stack(dev[map_column])
x_test = np.stack(test[map_column])

print("Wall time: {:.2f} seconds".format(time.time() - start))
# sanity check
# expected: (#rows, xdim, ydim)
print(x_train.shape)

In [ ]:
# create tf dataset
start = time.time()

x_train = tf.data.Dataset.from_tensor_slices(x_train)
x_val = tf.data.Dataset.from_tensor_slices(x_val)
x_test = tf.data.Dataset.from_tensor_slices(x_test)

print("Wall time: {:.2f} seconds".format(time.time() - start))
# sanity check
# expected: shape=(224,224,1,1) dtype=int
x_train.element_spec

In [ ]:
# # expand tensor and create dummy dimension at axis 3
# # images in greyscale, so no channel dimension
# start = time.time()

# x_train = tf.expand_dims(x_train, axis=3, name=None)
# x_val = tf.expand_dims(x_val, axis=3, name=None)
# x_test = tf.expand_dims(x_test, axis=3, name=None)

# print("Wall time: {:.2f} seconds".format(time.time() - start))
# # sanity check
# # expected: TensorShape([#rows, xdim, ydim, 1])
# x_train.shape

In [ ]:
# prepare labels for supervised learning
# note: make sure labels are integers if using sparse categorical cross entropy
start = time.time()

y_train = np.asarray(train['detectLabels']).astype(np.uint8)
y_val = np.asarray(dev['detectLabels']).astype(np.uint8)
y_test = np.asarray(test['detectLabels']).astype(np.uint8)

print("Wall time: {:.2f} seconds".format(time.time() - start))
# sanity check
type(y_train[0])

#### Model

In [ ]:
# define model architecture
# REMOVE RESIZING FOR 224x224 MODELS
# REMOVE RESCALING FOR THIN DATASETS

model = models.Sequential()
model.add(layers.experimental.preprocessing.Rescaling(scale=1./2., input_shape=x_train.shape[1:]))
model.add(layers.experiemntal.preprocessing.Resizing(224, 224, interpolation='nearest'))
model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D(3))
model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D(3))
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='sigmoid'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))
model.summary()

In [ ]:
# set model optimizer and metrics
opt = optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])

In [ ]:
# set up early stopping and save model checkpoints
early_stopping = callbacks.EarlyStopping(monitor='loss', patience=10)
checkpoint = callbacks.ModelCheckpoint(f'{model_path}/{model_id}-{data_id}{note}', monitor='val_loss', mode='min', 
                                       verbose=0, save_best_only=True, save_weights_only=False)

In [ ]:
# run model
start = time.time()

history = model.fit(x_train, y_train, batch_size=64, validation_data=(x_val, y_val),
                    epochs=100, callbacks=[early_stopping, checkpoint], verbose=1)

print("Wall time: {:.2f} seconds".format(time.time() - start))

In [ ]:
# visualize accuracy and loss history
fig, axs = plt.subplots(2, 1, figsize=(15,15))

axs[0].plot(history.history['loss'])
axs[0].plot(history.history['val_loss'])
axs[0].title.set_text('Training Loss vs Validation Loss')
axs[0].legend(['Train', 'Val'])

axs[1].plot(history.history['accuracy'])
axs[1].plot(history.history['val_accuracy'])
axs[1].title.set_text('Training Accuracy vs Validation Accuracy')
axs[1].legend(['Train', 'Val'])

#### Model results

In [ ]:
# load best model from checkpoint
best_model = keras.models.load_model(f'{model_path}/{model_id}-{data_id}{note}')

In [ ]:
# compute model results on test set
start = time.time()
results = best_model.evaluate(x_test, y_test)
print("Wall time: {:.2f} seconds".format(time.time() - start))
print(results)

In [ ]:
# generate predictions for model analysis
start = time.time()
y_pred = model.predict(x_test)
y_max = np.argmax(y_pred, axis=1).astype(np.uint8)
predictions = [y_max, y_pred]
print("Wall time: {:.2f} seconds".format(time.time() - start))

In [ ]:
# save predictions
# save to local instance
with open(f'{result_path}/{model_id}-{data_id}{note}.pkl', "wb") as f:
    pickle.dump(predictions, f)

In [ ]:
# save predictions
# save to S3
pkl_obj = pickle.dump(predictions)
s3.Object(bucket_name, f'results/{model_id}-{data_id}{note}.pkl').put(Body=pkl_obj)

In [ ]:
# save best model to S3
s3_save_keras_model(best_model, f'{model_id}-{data_id}{note}')

In [ ]:
# plot confusion matrix
helper.plot_confusion_matrix(y_test, y_max, mode='detect', normalize=True)

In [ ]:
# plot confusion matrix counts
helper.plot_confusion_matrix(y_test, y_max, mode='detect', normalize=False)